### ChainMap

Recall chain from itertools...

In [1]:
from itertools import chain
l1 = [1, 2, 3]
l2 = (4, 5, 6)
l3 = (x for x in (7, 8, 9))

for e in chain(l1, l2, l3):
    print(e)

1
2
3
4
5
6
7
8
9


It makes it look like we have a single iterable, but really it just chained them one after the other!

collections.ChainMap serves a similar purpose - chaining dictionaries (or mapping types in general)

Suppose we have 3 dictionaries:
- d1, d2, d3

We can import ChainMap from the collections module, and chain the dictionaries together!
- This requires no extra storage (nothing is copied)
- Note! Mutating elements in the chain may affect underlying dicts
- The ChainMap object also sees changes in the underlying dictionary!
- It haves more like a dictionary view (but is updatable!)

This is very different from the following:

In [ ]:
d = {**d1, **d2, **d3}

As this requires extra storage, it is essentially a shallow copy/merge, and it does not see changes in underlying dictionaries

#### Reading Keys from a Chain

There's an added complexity chaining maps that we do not have with iterables

First, the resulting chain should itself be a map -> no repeated keys!

So if we have two dictionaries with similar keys, the first instance of the key encountered in the chain shall be used to assign a value to the ChainMap key value

See code for an example of this

This is again unlike unpacking multiple dictionaries into a dictionary, as in that instance the last occuring key instance is what dictates the value of the key

**Be Careful!** Unlike a dict, there is no guarantee of key order when iterating a ChainMap

#### Think of it as a Parent-Child Relationship

Suppose we have ChainMap(d1, d2, d3)

d1 is the child, while d2 and d3 are the parents

When we ChainMap, d2 overrides any values in d3, while d1 overrides any values in d2 or d3

In fact, there are attributes to deal with this explicitly
- d.parents -> returns a ChainMap containing the parent elements only
- d.new_child(d4) -> adds d4 to the front of the chain (or bottom of the hierarchy)

#### Additional ways to update the Chain

The .maps property returns a (mutable) list of all the maps in the chain
- The order of the list is the same as the child -> parents hierarchy
- Since the list is mutable, we can modify the chain by removing, deleting, inserting, and appending other maps

#### Mutating Maps via the ChainMap

The ChainMap is mutable -> we already saw we could add and remove maps from the chain

We can also mutate the key/value pairs in the map itself

So if we had this:

In [ ]:
d = ChainMap(d1, d2)

We could do this:

In [ ]:
d[key] = new_value

**BUT** these mutations only affect the child(first) map!

So if we had the following:

In [ ]:
d1 = {'a': 1, 'b': 2}
d2 = {'a': 20, 'c': 3}
d = ChainMap(d1, d2)

And we did:

In [ ]:
d['a'] = 100

Then we would find

In [ ]:
d1 = {'a': 100, 'b': 2}
d2 = {'a': 20, 'c': 3}

So the parent dictionary is not affected!

The same is true with keys not in the child dict...

Suppose we did this:

In [ ]:
d['c'] = 200

We would find that

In [ ]:
d1 = {'a': 100, 'b': 2, 'c': 200}
d2 = {'a': 20, 'c': 3}

If we delete a key, it will only be deleted from the child dict as well!

And if you try to delete a key not in the child dict, but within the chain, you get a KeyError exception!

#### Code Examples

In [5]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d3 = {'e': 5, 'f': 6}

In [6]:
d = {**d1, **d2, **d3}

In [7]:
print(d)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}


In [8]:
d = {}
d.update(d1)
d.update(d2)
d.update(d3)
print(d)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}


In [9]:
from collections import ChainMap

In [10]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d3 = {'e': 5, 'f': 6}

In [11]:
d = ChainMap(d1, d2, d3)

In [12]:
print(d)

ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})


In [13]:
isinstance(d, dict)

False

In [14]:
d['a']

1

In [15]:
d['f']

6

In [16]:
for key, value in d.items():
    print(key, value)

e 5
f 6
c 3
d 4
a 1
b 2


In [17]:
d1 = {'a': 1, 'b': 2}
d2 = {'b': 20, 'c': 3}
d3 = {'c': 30, 'd': 4}

In [18]:
d = {**d1, **d2, **d3}

In [19]:
d

{'a': 1, 'b': 20, 'c': 30, 'd': 4}

In [20]:
d = ChainMap(d1, d2, d3)

In [21]:
d['b']

2

In [22]:
d

ChainMap({'a': 1, 'b': 2}, {'b': 20, 'c': 3}, {'c': 30, 'd': 4})

In [23]:
for k, v in d.items():
    print(k, v)

c 3
d 4
b 2
a 1


In [24]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d3 = {'e': 5, 'f': 6}
d = ChainMap(d1, d2, d3)

In [25]:
d['z'] = 100

In [26]:
print(d)

ChainMap({'a': 1, 'b': 2, 'z': 100}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})


In [27]:
d1

{'a': 1, 'b': 2, 'z': 100}

In [28]:
for key in d:
    print(key)

e
f
c
d
a
b
z


In [29]:
d['c'] = 300

In [30]:
d['c']

300

In [31]:
d

ChainMap({'a': 1, 'b': 2, 'z': 100, 'c': 300}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})

In [32]:
del d['e']

KeyError: "Key not found in the first mapping: 'e'"

In [33]:
del d['c']

In [34]:
d['c']

3

In [35]:
d

ChainMap({'a': 1, 'b': 2, 'z': 100}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})

In [36]:
d3['x'] = 500

In [38]:
d['x']

500

In [41]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d = ChainMap(d1, d2)
print(id(d))

140614352448336


In [42]:
d3 = {'d': 400, 'e': 5}

In [44]:
d = ChainMap(d, d3)
print(id(d))

140614352471248


In [45]:
d

ChainMap(ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4}), {'d': 400, 'e': 5})

In [46]:
d['d']

4

In [47]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d = ChainMap(d1, d2)

In [48]:
d3 = {'d': 400, 'e': 5}
d = ChainMap(d3, d)

In [49]:
d

ChainMap({'d': 400, 'e': 5}, ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4}))

In [50]:
d['d']

400

In [51]:
d1 = {'a': 1, 'b': 2}
d2 = {'c': 3, 'd': 4}
d = ChainMap(d1, d2)

In [52]:
d3 = {'d': 400, 'e': 5}
d = d.new_child(d3)

In [53]:
d

ChainMap({'d': 400, 'e': 5}, {'a': 1, 'b': 2}, {'c': 3, 'd': 4})

In [54]:
d.parents

ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4})

In [56]:
type(d.maps), d.maps

(list, [{'d': 400, 'e': 5}, {'a': 1, 'b': 2}, {'c': 3, 'd': 4}])

In [57]:
d3 = {'e': 5, 'f': 6}

In [58]:
d

ChainMap({'d': 400, 'e': 5}, {'a': 1, 'b': 2}, {'c': 3, 'd': 4})

In [59]:
d.maps.append(d3)

In [60]:
d

ChainMap({'d': 400, 'e': 5}, {'a': 1, 'b': 2}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})

In [61]:
d['f']

6

In [62]:
d.maps

[{'d': 400, 'e': 5}, {'a': 1, 'b': 2}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6}]

In [63]:
del d.maps[0]

In [64]:
d.maps

[{'a': 1, 'b': 2}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6}]

In [65]:
d

ChainMap({'a': 1, 'b': 2}, {'c': 3, 'd': 4}, {'e': 5, 'f': 6})

In [67]:
config = {
    'host': 'prod.deepdive.com',
    'port': 5432,
    'database': 'deepdive',
    'user_id': '$pg_user',
    'user_pwd': '$pg_pwd'  
}

In [69]:
local_config = ChainMap({}, config)

In [70]:
list(local_config.items())

[('host', 'prod.deepdive.com'),
 ('port', 5432),
 ('database', 'deepdive'),
 ('user_id', '$pg_user'),
 ('user_pwd', '$pg_pwd')]

In [71]:
local_config['user_id'] = 'test'
local_config['user_pwd'] = 'test'

In [72]:
list(local_config.items())

[('host', 'prod.deepdive.com'),
 ('port', 5432),
 ('database', 'deepdive'),
 ('user_id', 'test'),
 ('user_pwd', 'test')]

In [73]:
local_config

ChainMap({'user_id': 'test', 'user_pwd': 'test'}, {'host': 'prod.deepdive.com', 'port': 5432, 'database': 'deepdive', 'user_id': '$pg_user', 'user_pwd': '$pg_pwd'})

In [74]:
config

{'host': 'prod.deepdive.com',
 'port': 5432,
 'database': 'deepdive',
 'user_id': '$pg_user',
 'user_pwd': '$pg_pwd'}